# Reinforcement Learning: An Introduction

In [ ]:
%%javascript
IPython.notebook.kernel.execute('NOTEBOOK_NAME="' + IPython.notebook.notebook_name + '"')

In [ ]:
KERNEL_VENV = '.venv'
KERNEL_NAME = 'rl'
KERNEL_DISPLAY_NAME = 'RL-venv'

In [ ]:
%%bash -s "$KERNEL_VENV" "$KERNEL_NAME" "$KERNEL_DISPLAY_NAME"
# Create a venv and a kernel for the notebook
KERNEL_VENV=$1
KERNEL_NAME=$2
KERNEL_DISPLAY_NAME=$3

if [ "$(. "$KERNEL_VENV/bin/activate" 2> /dev/null && which python)" != "$(pwd)/$KERNEL_VENV/bin/python" ]; then
    python -m venv "$KERNEL_VENV" --prompt "$KERNEL_DISPLAY_NAME" --system-site-packages;
    echo "Created virtual environment: '$(pwd)/$KERNEL_VENV'"
    OUTPUT_NOT_EMPTY=1
fi

if ! jupyter kernelspec list | grep -q $KERNEL_NAME; then
    (. "$KERNEL_VENV/bin/activate"; python -m ipykernel install --user --name=$KERNEL_NAME --display-name="$KERNEL_DISPLAY_NAME")
    echo "## Please refresh page and select: Kernel > Change kernel > $KERNEL_DISPLAY_NAME ##"
    OUTPUT_NOT_EMPTY=1
fi

if [ -z "$OUTPUT_NOT_EMPTY" ]; then echo "No news is good news!"; fi

In [ ]:
! git status

In [ ]:
! mv Reinforcement

In [ ]:
COMMIT_MESSAGE = input("Next commit message: ")

In [ ]:
%%bash -s "$NOTEBOOK_NAME" "$KERNEL_DISPLAY_NAME" "$COMMIT_MESSAGE"
# Create a venv and a kernel for the notebook
NOTEBOOK_NAME=$1
KERNEL_DISPLAY_NAME=$2
COMMIT_MESSAGE=$3

SAVED_KERNEL=$(grep -A4 \"kernelspec\" "$NOTEBOOK_NAME" | grep -Po "(?<=\"display_name\": \")[^,\"]+")
if [ "$SAVED_KERNEL" = "$KERNEL_DISPLAY_NAME" ]; then
#     if git checkout develop; then
#     else
#     fi
    echo foo
    mkdir -p .backups
    cp "$NOTEBOOK_NAME" .backups/
    python -m nbconvert --clear-output "$NOTEBOOK_NAME"
    git add "$NOTEBOOK_NAME"
    git commit -m "$COMMIT_MESSAGE"
    cp ".backups/$NOTEBOOK_NAME" ./
#     git push
else
    echo bar
fi


In [ ]:
! sed -n -e '/"kernelspec"/,/'$(printf "\x7d")'/p' *.ipynb | grep "\"name\": \"{KERNEL_NAME}\""

In [ ]:
x = input('LOL')
%%bash
echo 

In [ ]:
!echo $(printf '\x42')

In [ ]:
! . '{kernel_venv}/bin/activate'; which python

In [ ]:
! jupyter kernelspec list | grep '{x}'

In [ ]:
x = ''

In [ ]:
# Pypi libraries
import numpy as np
import matplotlib.pyplot as plt

# Matplotlib configs
# %config InlineBackend.figure_formats = ['svg']
# plt.rcParams['figure.dpi'] = 300

In [ ]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
!echo $(pwd)/{nb_name}

In [ ]:
__file__
# python -m nbconvert --clear-output .ipynb

# Chapter 2

## 2.3 - The 10-armed Testbed

In [ ]:
class ArmedBandit:
    def __init__(self,arms,runs):
        self.arms = arms
        self.runs = runs
        self.runs_range = np.arange(self.runs)
        
        self.action_values = np.random.normal(0,1,size=(self.arms,self.runs))
        self.optimal_action = self.action_values.argmax(axis=0)
    
    def step(self,action):
        return np.random.normal(self.action_values[action,self.runs_range],1)

In [ ]:
class EpsilonPolicy:
    def __init__(self,actions,runs,epsilon):
        self.actions = actions
        self.runs = runs
        self.runs_arange = np.arange(runs)
        self.epsilon = epsilon
        
        self.action_reward = np.zeros((actions,runs))
        self.action_count = np.zeros((actions,runs))
        self.prev_action = np.zeros(runs,dtype=int)
    
    def act(self):
        # action_type: 0=random, 1=greedy
        action_type = np.random.rand(self.runs) > self.epsilon
        
        # random actions
        self.prev_action[~action_type] = np.random.randint(self.actions,size=sum(~action_type))
        # greedy actions: argmax estimate action values via sample average (replace 0 counts by 1 to avoid div0 error)
        self.prev_action[action_type] = (self.action_reward[:,action_type] / \
                                         (self.action_count[:,action_type] + (self.action_count[:,action_type] == 0)) \
                                        ).argmax(axis=0)
        return self.prev_action
    
    def update(self,reward):
        self.action_reward[self.prev_action,self.runs_arange] += reward
        self.action_count[self.prev_action,self.runs_arange] += 1

In [ ]:
class PolicyAnalyser:        
    def __init__(self,episode_length):
        self.episode_length = episode_length
        self.labels = []
        self.reward_history = np.zeros((0,episode_length))
        self.optimal_history = np.zeros((0,episode_length))
        
    def create(self,label,env,policy):
        self.env = env
        self.policy = policy
        
        self.step = 0
        self.labels.append(label)
        self.reward_history = np.concatenate((self.reward_history,np.zeros((1,self.episode_length))))
        self.optimal_history = np.concatenate((self.optimal_history,np.zeros((1,self.episode_length))))
        
    def log(self,reward):
        self.reward_history[-1,self.step] = reward.mean()
        self.optimal_history[-1,self.step] = (self.policy.prev_action == self.env.optimal_action).sum()/self.env.runs
        self.step += 1
    
    def plot(self):
        plt.figure()
        plt.plot(self.reward_history.transpose(),label=self.labels,linewidth=0.75)
        plt.legend()
        
        plt.figure()
        plt.plot(self.optimal_history.transpose(),label=self.labels,linewidth=1)
        plt.legend()

In [ ]:
def run_bandit_episode(env,policy,datalog,episode_length):
    for i in range(episode_length):
        action = policy.act()
        reward = env.step(action)
        policy.update(reward)
        datalog.log(reward)

arms = 10
episode_length = 1000
runs = 2000
epsilon = [0,0.01,0.1]

datalog = PolicyAnalyser(episode_length)
for eps in epsilon:
    env = ArmedBandit(arms,runs)
    policy = EpsilonPolicy(arms,runs,eps)
    datalog.create(f'eps={eps}',env,policy)
    run_bandit_episode(env,policy,datalog,episode_length)

In [ ]:
datalog.plot()

## Exercice 2.5 - Nonstationary Bandit Problem

In [ ]:
# Non-stationnary Armed Bandit
class NSArmedBandit(ArmedBandit):
    # Armed Bandit problem with action values initialised to 0
    # and a random walk every step
    def __init__(self,action_value_dev,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self._step = super().step
        self.action_values = np.zeros((self.arms,self.runs))
        self.action_value_dev = action_value_dev
        
    def step(self,*args,**kwargs):
        self.action_values = np.random.normal(self.action_values,self.action_value_dev)
        self.optimal_action = self.action_values.argmax(axis=0)
        return self._step(*args,**kwargs)

In [ ]:
random_walk_std_dev = 0.01

datalog = PolicyAnalyser(episode_length)
for eps in epsilon:
    env = NSArmedBandit(random_walk_std_dev,arms,runs)
    policy = EpsilonPolicy(arms,runs,eps)
    datalog.create(f'eps={eps}',env,policy)
    run_bandit_episode(env,policy,datalog,episode_length)

In [ ]:
datalog.plot()